In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 3, Finished, Available, Finished)

#### _**Ler arquivos do Lakehouse**_

In [2]:
df = spark.read.parquet("Files/raw-data/vendas.parquet")

display(df)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2025104e-70f8-41eb-b639-ceb3e830bfe1)

#### _**Verificação da qualidade dos dados**_
* Colunas disponíveis X Colunas relevantes
* Verificação de valores nulos
* Verificação de duplicatas
* Verificação de tipos de colunas
* Padronização e verificação dos nomes das colunas
* Verificação de consistência e integridade dos Dados
* Distribuição de dados e outliers
* Verificação de formatos de dados específicos

#### _**Verificar estrutura da tabela como colunas disponíveis, nome das colunas e tipos**_

In [3]:
df.printSchema()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 5, Finished, Available, Finished)

root
 |-- Categoria: string (nullable = true)
 |-- SubCategoria: string (nullable = true)
 |-- Modelo: string (nullable = true)
 |-- Cor: string (nullable = true)
 |-- Tamanho: string (nullable = true)
 |-- Preço Custo Unit: string (nullable = true)
 |-- Data Pedido: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Região: string (nullable = true)
 |-- Pedidos Qtd: string (nullable = true)
 |-- Total Venda: string (nullable = true)
 |-- ID_VENDA: string (nullable = true)



In [4]:
df_verificar = df.select("Pais").distinct()

df_verificar.show()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 6, Finished, Available, Finished)

+------+
|  Pais|
+------+
|Brasil|
+------+



Mantém linhas únicas, removendo todas as duplicatas:
```
distinct()
```

#### **_Verificar valores nulos_**

In [5]:
df_verificar_nulos = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

display(df_verificar_nulos)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8df6794b-6a52-4976-94d5-135aea8192a2)

Função agregada que soma os valores de cada coluna:
```
sum()
```

Verifica se os valores de uma coluna são nulos:
```
isNull()
```

Renomeia a coluna resultante da operação:
```
alias()
```


#### _**Verificar se há valores duplicados em um campo que deveria ser único**_

In [6]:
df_verificar_duplicatas = df.groupBy("ID_VENDA").count().filter(col("count") > 1)

display(df_verificar_duplicatas)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8678f2de-f572-43d5-8e58-d93ca5c0bdf8)

#### _**Verificar valores máximos e minímos para identificar Outliers**_

In [7]:
df_verificar_max = df.select("Preço Custo Unit").distinct().orderBy(col("Preço Custo Unit").desc())

display(df_verificar_max)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 222165d3-653d-48b4-82ad-2f5ad2d9fe33)

Ordena os valores da coluna especificada em ordem decrescente.
```
orderBy(col("").desc())
```

Ordena os valores da coluna especificada em ordem crescente.
```
orderBy(col("").asc())
```

#### _**Verificar atributos específicos**_

In [8]:
df_verificar_atributos = df.groupBy("Tamanho").count().show()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 10, Finished, Available, Finished)

+------------+-----+
|     Tamanho|count|
+------------+-----+
|          54|  311|
|          42|  542|
|          52|  424|
|        null| 5206|
|          70|  175|
|          46|  544|
|     Pequeno|  378|
|          60|  282|
|       Médio|  463|
|          38|  458|
|          40|  179|
|          44|  678|
|      Grande|  451|
|          58|  455|
|Extra Grande|  208|
|          48|  616|
|          56|   95|
|          62|  242|
|          50|  277|
+------------+-----+



#### _**Remover colunas desnecessárias**_

In [9]:
df = df.drop("Pais")

df.printSchema()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 11, Finished, Available, Finished)

root
 |-- Categoria: string (nullable = true)
 |-- SubCategoria: string (nullable = true)
 |-- Modelo: string (nullable = true)
 |-- Cor: string (nullable = true)
 |-- Tamanho: string (nullable = true)
 |-- Preço Custo Unit: string (nullable = true)
 |-- Data Pedido: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Região: string (nullable = true)
 |-- Pedidos Qtd: string (nullable = true)
 |-- Total Venda: string (nullable = true)
 |-- ID_VENDA: string (nullable = true)



#### _**Tratar dados nulos**_

In [10]:
# Preencher dados nulos.

df = df.na.fill("NaN")

display(df)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 92198317-2c12-4cec-a2f8-4efa5e1ada5c)

#### _**Alterar formato dos dados**_

In [11]:
df = df.withColumn("Preço Custo Unit", regexp_replace(col("Preço Custo Unit"), ",", "."))

df = df.withColumn("Total Venda", regexp_replace(col("Total Venda"), ",", "."))

df = df.withColumn("Data Pedido", to_date(col("Data Pedido"), "dd/MM/yyyy"))

display(df)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, afd6b322-0753-46c6-8d60-9f5ffe9176a5)

Cria uma nova coluna ou substitui uma coluna existente:
```
withColumn()
```

Substitui todas as ocorrências especificadas:
```
regexp_replace(col(""), ",", ".")
```

Converte a coluna para o tipo de dado Date usando o formato "dd/MM/yyyy".
```
to_date(col(""), "dd/MM/yyyy")
```

In [12]:
df.printSchema()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 14, Finished, Available, Finished)

root
 |-- Categoria: string (nullable = false)
 |-- SubCategoria: string (nullable = false)
 |-- Modelo: string (nullable = false)
 |-- Cor: string (nullable = false)
 |-- Tamanho: string (nullable = false)
 |-- Preço Custo Unit: string (nullable = false)
 |-- Data Pedido: date (nullable = true)
 |-- Estado: string (nullable = false)
 |-- UF: string (nullable = false)
 |-- Cidade: string (nullable = false)
 |-- Região: string (nullable = false)
 |-- Pedidos Qtd: string (nullable = false)
 |-- Total Venda: string (nullable = false)
 |-- ID_VENDA: string (nullable = false)



#### _**Alterar tipos dos dados**_

In [13]:
df = df.withColumn("Categoria", col("Categoria").cast("string")) \
       .withColumn("SubCategoria", col("SubCategoria").cast("string")) \
       .withColumn("Modelo", col("Modelo").cast("string")) \
       .withColumn("Cor", col("Cor").cast("string")) \
       .withColumn("Tamanho", col("Tamanho").cast("string")) \
       .withColumn("Preço Custo Unit", col("Preço Custo Unit").cast("double")) \
       .withColumn("Estado", col("Estado").cast("string")) \
       .withColumn("UF", col("UF").cast("string")) \
       .withColumn("Cidade", col("Cidade").cast("string")) \
       .withColumn("Região", col("Região").cast("string")) \
       .withColumn("Pedidos Qtd", col("Pedidos Qtd").cast("integer")) \
       .withColumn("Total Venda", col("Total Venda").cast("double")) \
       .withColumn("ID_VENDA", col("ID_VENDA").cast("integer"))

df.printSchema()

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 15, Finished, Available, Finished)

root
 |-- Categoria: string (nullable = false)
 |-- SubCategoria: string (nullable = false)
 |-- Modelo: string (nullable = false)
 |-- Cor: string (nullable = false)
 |-- Tamanho: string (nullable = false)
 |-- Preço Custo Unit: double (nullable = true)
 |-- Data Pedido: date (nullable = true)
 |-- Estado: string (nullable = false)
 |-- UF: string (nullable = false)
 |-- Cidade: string (nullable = false)
 |-- Região: string (nullable = false)
 |-- Pedidos Qtd: integer (nullable = true)
 |-- Total Venda: double (nullable = true)
 |-- ID_VENDA: integer (nullable = true)



O método cast é utilizado para converter o tipo de dados da coluna:
```
.cast("double")
```

#### _**Alterar nomes das colunas**_

In [14]:
df = df.withColumnRenamed("Categoria", "categoria") \
    .withColumnRenamed("SubCategoria", "subcategoria") \
    .withColumnRenamed("Modelo", "modelo") \
    .withColumnRenamed("Cor", "cor") \
    .withColumnRenamed("Tamanho", "tamanho") \
    .withColumnRenamed("Preço Custo Unit", "preco_custo_unit") \
    .withColumnRenamed("Data Pedido", "data_pedido") \
    .withColumnRenamed("Estado", "estado") \
    .withColumnRenamed("UF", "uf") \
    .withColumnRenamed("Cidade", "cidade") \
    .withColumnRenamed("Região", "regiao") \
    .withColumnRenamed("Pedidos Qtd", "pedidos_qtd") \
    .withColumnRenamed("Total Venda", "total_venda") \
    .withColumnRenamed("ID_VENDA", "id_venda")

display(df)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 14711e7d-c522-4d65-a0e1-46be99383ae5)

Função que renomeia uma coluna existente no DataFrame:
```
withColumnRenamed()
```

#### _**Converter para delta table**_

In [15]:
display(df)

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 318f2a04-ad4e-4367-aa96-a973350038db)

In [16]:
df.write.format("delta").mode("append").save("abfss://85718698-872f-4ed2-b22e-50273ba4f3aa@onelake.dfs.fabric.microsoft.com/08722e8f-8e40-4011-803a-1994cd0a4f9c/Tables/Vendas")

StatementMeta(, 042de696-83b9-4833-ab89-29d44f0431ef, 18, Finished, Available, Finished)

**append**: Adiciona novos dados ao final.

**overwrite**: Substitui completamente os dados existentes.

**ignore**: Não faz alterações se o destino existir.

**error**: Gera um erro se o destino existir.